# Constantin

我们有时候希望直接使用 RTL 测试不同参数下的性能表现，然而，每调整一次参数就进行一次编译非常浪费时间（尽管 RTL 的绝大部分都没有变化，verilator 仍然需要完成全量的编译）。有没有什么办法可以不编译就改变参数呢？

Sometimes we want to directly test the performance under different parameters using RTL. However, it is very time-consuming to compile every time we adjust a parameter (although most of the RTL does not change, verilator still needs to complete a full compilation). Is there any way to change parameters without compiling?

我们提出 Constantin，它基于 DPI-C 接口，使用 C++ 函数和 Chisel 的 BlackBox 机制在运行初始化时对参数进行配置。

We present Constantin, which is based on the DPI-C interface and uses C++ functions and Chisel's BlackBox mechanism to configure parameters during runtime initialization.

![Overview](../images/03-performance/04-constantin/overview-en.png)

使用 Constantin 代替一个 scala 参数大致如下所示：

To replace a scala parameter with Constantin, it looks roughly like this:

```scala
/* *** w/o Constantin *** */
val enableSomeModule = WireInit(false.B) // change to true.B, re-compile and re-run

/* *** w/ Constantin *** */
// in RTL
val enableSomeModule = WireInit(Constantin.createRecord("enableSomeModule", initValue = false))
// in constantin.txt
enableSomeModule 0 // change to 1, re-run
```

要启用 Constantin，需要在编译 emu 时使用 `WITH_CONSTANTIN=1` 选项。

To enable Constantin, you need to use the `WITH_CONSTANTIN=1` option when compiling emu.

目前 Constantin 暂未实现 emu 参数化，其配置文件必须位于 `${NOOP_HOME}/build/constantin.txt`。

Currently, Constantin does not present as an emu argument, its configuration file must be located at `${NOOP_HOME}/build/constantin.txt`.

下面的例子使用 Constantin 控制分支预测器的开关，您可以运行该单元格来对比开和关的差异。

The following example uses Constantin to control the switch of the branch predictor. You can run this cell to compare the differences between on and off.

In [ ]:
%%bash
cd .. && source env.sh

mkdir -p ${NOOP_HOME}/build

# run with default parameter
rm -f ${NOOP_HOME}/build/constantin.txt || true
$(get_asset emu-precompile/emu-constantin) \
    -i $(get_asset workload/coremark-2-iteration.bin) \
    -C 10000 \
    --no-diff \
    2>/dev/null

# run with Bpu turned off (falls back to static not-taken prediction)
echo "enableUbtb 0" > ${NOOP_HOME}/build/constantin.txt
$(get_asset emu-precompile/emu-constantin) \
    -i $(get_asset workload/coremark-2-iteration.bin) \
    -C 10000 \
    --no-diff \
    2>/dev/null

## Autosolving

我们实现的 Constantin 还提供了一个 Autosolving 功能，其可以自动找到当前微架构下的最佳参数配置。

We also implemented Autosolving for Constantin, which can automatically find the best parameter configuration under the current microarchitecture.

要使用 Autosolving，需要在 Constantin 的配置中将其启用（参考 `./04-autosolving.patch`），并开启 `WITH_CONSTANTIN=1` 编译 emu。

To use Autosolving, you need to enable it in the Constantin configuration (refer to `./04-autosolving.patch`), and compile emu with `WITH_CONSTANTIN=1`.

在开启 Autosolving 后，emu 将从 stdin 而非 txt 文件中读取 Constantin 配置，这样就可以使用我们的 python 脚本自动运行特定配置的 emu 并尝试找到最优配置。

After enabling Autosolving, emu will read the Constantin configuration from stdin instead of a txt file, allowing us to use our python script to automatically run emu with specific configurations and try to find the optimal configuration.

您需要为脚本提供一个配置文件（参考 `04-autosolving-config.json`），包括：
- 可配置的参数描述
  - 参数名
  - 位宽
  - 初始值
- 优化目标
  - 性能计数器名
  - 策略（最小化、最大化）
  - 基线
- 遗传算法参数
- emu 运行参数
  - workload
  - 最大指令数
  - 线程数

You need to provide a configuration file for the script (refer to `04-autosolving-config.json`), including:
- Descriptions of configurable parameters
  - Parameter name
  - Bit width
  - Initial value
- Optimization goals
  - Performance counter name
  - Strategy (minimize, maximize)
  - Baseline
- Genetic algorithm parameters
- emu running parameters
  - workload
  - Maximum number of instructions
  - Number of threads

我们的脚本使用遗传算法进行参数探索，要实现其它算法（例如蚁群/粒子群优化）也是很方便的。

Our script uses genetic algorithm for parameter exploration, and it is also easy to implement other algorithms (such as ant colony/particle swarm optimization).

In [ ]:
%%bash
cd .. && source env.sh && cd - >/dev/null

echo === Patch ===
cat ./04-autosolving.patch

echo === Config ===
cat ./04-autosolving-config.json

echo === Run ===
mkdir -p ${NOOP_HOME}/build
cp $(get_asset emu-precompile/emu-autosolving) ${NOOP_HOME}/build/emu
python3 ${NOOP_HOME}/scripts/constantHelper.py ./04-autosolving-config.json